# GraphGuard

***Locate and find Classes in Apks with updated Obfuscation Mapping***


Processing Steps:
1. String Strategy (Classes + Methods) \
  Strategy using 2 different variants only involving Strings:
  * String Counter: used in Classes and Methods and try to find exact matching counter.
  * Unique Strings: Match Classes by identifying Strings used only in this single Class.



2. Structure Strategy (Classes) \
  Strategy that enumerates all classes and tries to find an optimal match by using the following (weighted) criteria:
  * Modifiers of class
  * Modifiers, Parameters, Parameter Types and Return Types of Methods
  * Number and Types of Fields.



3. Method Strategy (Methods) \
  Strategy that uses already matched classes and tries to find an optimal Method Match in the class by using the following (weighted) criteria:
  * Modifiers
  * Return Type, Parameter Types
  * Bytecode Length
  * References to and from


4. Field Strategy (Fields) \
  Strategy that uses already matched classes and tries to find an optimal Field Match in the class by using the following (weighted criteria)
  * By Type if a matching Type has been found
  * By weighted sub-criteria such as:
    * Access Flags
    * Size
    * Number of reads & Number of writes (references)

***If you use Akrolyb, please have a look at [Akrolyb Interoptability](#Interoptability-with-Akrolyb)***

In [1]:
%matplotlib notebook

%load_ext autoreload

from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

In [2]:
%autoreload
from collections.abc import Iterable

from core.start import process_files
from utils.formats import *

from core.accumulator import Accumulator
from core.decs import *

from strategies import\
    strings as strings_strategy,\
    methods as methods_strategy,\
    structures as structures_strategy,\
    fields as fields_strategy

from core.strategy_handler import StrategyHandler, FLAG_CLASS, FLAG_METHOD, FLAG_FIELD

from utils import generate
from utils import io_akrolyb

# Loading Androguard

The following code loads the files and starts Androguard

It should support multiprocessing, however the Pipe communication seems to break when transmitting the processed Androguard Objects. I suspect the Object is simply too big for Pickle to serialize or another component in the transmitting chain.

In [3]:
AG_SESSION_FILE = "./Androguard.ag"
MULTIPROCESS_FILES = False  # Currently not working due to serialization issues

apk_dir = "../../../Downloads/"

# APK Files to load
file_paths = (
    "com.snapchat.android_11.3.0.58-2113_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "com.snapchat.android_11.3.5.66-2114_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk"
)


file_paths = (
    "com.instagram.android_164.0.0.46.123-252055936_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "com.instagram.android_164.0.0.46.123-252055925_minAPI19(armeabi-v7a)(420,400,360,480dpi)_apkmirror.com.apk"
)
file_paths = tuple(map(lambda apk_name: apk_dir + apk_name, file_paths))

In [4]:
(a, d, dx), (a2, d2, dx2) = process_files(*file_paths, MULTIPROCESS_FILES)

Loading Session from Apk at ../../../Downloads/com.instagram.android_164.0.0.46.123-252055936_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


Loading Session from Apk at ../../../Downloads/com.instagram.android_164.0.0.46.123-252055925_minAPI19(armeabi-v7a)(420,400,360,480dpi)_apkmirror.com.apk


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


# List of Classes, Methods and Fields

Defining Elements that GraphGuard should try to match (obviously requires full class names).

GraphGuard has full Akrolyb support. Read about it at [Akrolyb Interoptability](#Interoptability-with-Akrolyb). Using this is optional. In case you don't want to use this functionality, do not run the few last cells that are related to Akrolyb Support and Automation


* For using Akrolyb Support:
    * Use CodeGen in Akrolyb to generate a Python-compatible tuple of declarations that you can copy paste into `named_c_decs`, `named_f_decs` and `named_f_decs`.
    * Specify the correct file for auto replacing values, the "declarations files".
    * Types of the variables:
        * `named_c_decs` in form of a `Dict[str, str]`
        * `named_m_decs` in form of a `Dict[str, MethodDec]`
        * `named_f_decs` in form of a `Dict[str, FieldDec]`


* For not using Akrolyb, and just simply try and match the values (without Auto-Replacing):
    * Leave named_c_decs, named_m_decs, named_f_decs empty.
    * Assign the declarations directly to c_decs, m_decs, f_decs further below by uncommenting the existing cell.
    * Types of the variables:
        * `c_decs` in form of a `Tuple[str]`
        * `m_decs` in form of a `Tuple[MethodDec]`
        * `f_decs` in form of a `Tuple[FieldDec]`

In [5]:
# Classes of the Methods and Fields are automatically merged with c_decs

named_c_decs = {
    'MAIN_ACTIVITY': 'com.instagram.mainactivity.MainActivity',
    'CLIPITEM': 'X.9lT',
    'COMMENT_LONGCLICK_HANDLER': 'X.3Sh',
    'DEVELOPEROPTIONSLAUNCHER': 'com.instagram.debug.devoptions.api.DeveloperOptionsLauncher',
    'DIRECT_VISUAL_MESSAGE_VIEW_CONTROLLER': 'com.instagram.direct.visual.DirectVisualMessageViewerController',
    'DM_GESTUREDETECTOR': 'X.3KP',
    'DM_HOLDER': 'X.5Bk',
    'DM_REPLY': 'X.4Ij',
    'DM_VOICEMESSAGE': 'X.53g',
    'DMVIDEOPLAYER': 'X.5Uy',
    'FEEDITEM': 'X.1vG',
    'FRAGMENT': 'androidx.fragment.app.Fragment',
    'HEADERGENERATOR': 'X.0SW',
    'HTTPREQUESTTASKFACTORY': 'X.0ru',
    'INAPP_NOTIFICATION': 'X.BiP',
    'IGTV_VIEWER_FRAGMENT': 'X.9Uw',
    'REALTIME_CLIENT_MANAGER': 'com.instagram.realtimeclient.RealtimeClientManager',
    'JACKSON_OBJ': 'X.129',
    'JACKSON_CREATOR': 'X.0j5',
    'LIVE_POST': 'X.2DP',
    'MAINFEED': 'X.1RC',
    'MEDIAITEM': 'X.1f4',
    'MEDIAJSON_HELPER': 'com.instagram.feed.media.Media__JsonHelper',
    'NOTIFICATION': 'X.1nR',
    'NOTIFICATION_HANDLER': 'X.2dM',
    'PHOTOTIMERCONTROLLER': 'X.2yZ',
    'POSTDETAIL': 'X.1ed',
    'POSTGESTURE_SIMPLEPOST': 'X.239',
    'POSTGESTURE_CAROUSELIMAGE': 'X.2CQ',
    'POSTGESTURE_CAROUSELVIDEO': 'X.2WQ',
    'POSTVIEW': 'X.1ei',
    'PROFILE': 'X.0lY',
    'PROFILEDETAIL': 'X.3B2',
    'PROFILEPIC_LONGCLICK': 'X.3B9',
    'PROXYGEN_JNI_HANDLER': 'com.facebook.proxygen.JniHandler',
    'REELVIDEOPLAYER': 'X.2MU',
    'REEL_VIEWER_FRAGMENT': 'com.instagram.reels.fragment.ReelViewerFragment',
    'RESPONSE_HANDLER': 'X.0ns',
    'NOTIFICATION_RUNNABLE': 'X.BiQ',
    'SERIALSCHEDULER': 'X.0iw',
    'SESSION': 'X.0Ne',
    'SIMPLE_EXECUTOR': 'X.4QM',
    'SWIPE_NAVIGATION_CONTAINER': 'com.instagram.ui.swipenavigation.SwipeNavigationContainer',
    'USER': 'X.1os',
    'VIDEOSETTINGSMANAGER': 'X.1hW',
    'VOLUME_INDICATOR': 'com.instagram.ui.widget.volume.VolumeIndicator'
}

In [6]:
named_m_decs = {
    'ACTIVITY_ON_DESTROY': MethodDec('com.instagram.mainactivity.MainActivity', 'onDestroy'),
    'CLIPITEM_CONSTRUCTOR': MethodDec('X.9lT', '<init>', 'X.2BL', 'X.9md', 'X.0Ne', 'com.instagram.clips.intf.ClipsViewerConfig', 'android.view.View', 'X.9PD', 'X.6Zo'),
    'COMMENT_ONLONGCLICK': MethodDec('X.3Sh', 'onLongPress', 'android.view.MotionEvent'),
    'DEVELOPEROPTIONSLAUNCHER_LOADANDLAUNCHDEVELOPEROPTIONS': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.1Gp', 'androidx.fragment.app.FragmentActivity', 'X.0Ne', 'android.os.Bundle'),
    'DEVELOPEROPTIONSLAUNCHER_LAUNCHPROJECTCORESWITCHERTOOL': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'launchProjectEncoreSwitcherTool', 'android.content.Context', 'androidx.fragment.app.FragmentActivity', 'X.0Ne'),
    'DEVELOPEROPTIONSLAUNCHER_LAUNCHSTORIESEXPERIMENTSWITCHERTOOL': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'launchStoriesExperimentSwitcherTool', 'android.content.Context', 'androidx.fragment.app.FragmentActivity', 'X.0Ne'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_PAUSE': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0C', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_RESUME': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_SETSTATE': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0B', skip_params=True),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_STARTVIEW': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A04', 'com.instagram.direct.visual.DirectVisualMessageViewerController'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_ADVANCETONEXT': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0G', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'boolean'),
    'DMGESTUREDETECTOR_ONLONGPRESS': MethodDec('X.3KP', 'onLongPress', 'android.view.MotionEvent'),
    'DMHOLDER_GETCURRENTDMMEDIA': MethodDec('X.5Bk', 'A00'),
    'DMREPLYCALLBACK_RUN': MethodDec('X.4Ij', 'A0M', 'X.4Ij', 'int'),
    'DMREPLY_SETSEEN': MethodDec('X.4Ij', 'A0J', 'X.4Ij'),
    'DMVIDEOPLAYER_SEEKTO': MethodDec('X.5Uy', 'A02', 'int', 'boolean'),
    'FEEDITEM_JSON_GENERATE': MethodDec('X.1vG', 'A00', 'X.0jN'),
    'FRAGMENT_GETACTIVITY': MethodDec('androidx.fragment.app.Fragment', 'getActivity'),
    'HEADERGENERATOR_GENERATEFROMDEVICEINFO': MethodDec('X.0SW', 'A00', 'android.content.Context'),
    'HTTPREQUESTTASKFACTORY_CREATEREQUEST': MethodDec('X.0ru', 'A03', 'java.lang.String', 'X.0Ne'),
    'INAPPNOTIFICATION_RUN': MethodDec('X.BiP', 'run'),
    'IGTVVIEWERFRAGMENT_GETMEDIA': MethodDec('X.9Uw', 'A00', 'X.9Uw'),
    'IGTVVIEWERFRAGMENT_UPDATESTATE': MethodDec('X.9Uw', 'A0C', 'X.9Uw'),
    'INDICATEACTIVITY_DO': MethodDec('com.instagram.realtimeclient.RealtimeClientManager', 'sendCommand', 'java.lang.String', 'java.lang.String', 'com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback'),
    'JACKSON_CLOSE': MethodDec('X.129', 'close'),
    'JACKSON_CREATEGENERATOR': MethodDec('X.0j5', 'A03', 'java.io.OutputStream', 'java.lang.Integer'),
    'JACKSON_JSONFACTORY': MethodDec('X.0j5', '<init>', 'X.Dw6'),
    'LIVE_POST_JSON_GENERATE': MethodDec('X.2DP', 'A00', 'X.0kE', 'X.2DQ'),
    'MAINACTIVITY_GETCHILDFRAGMENTMANAGER': MethodDec('com.instagram.mainactivity.MainActivity', 'ALw'),
    'MAINFEED_CONFIGUREACTIONBAR': MethodDec('X.1RC', 'configureActionBar', 'X.1LS'),
    'MEDIAITEM_JSON_GENERATE': MethodDec('X.1f4', 'A01', 'X.0jN', 'boolean'),
    'MEDIA_JSON_GENERATE': MethodDec('com.instagram.feed.media.Media__JsonHelper', 'A00', 'X.0kE', 'X.1f4'),
    'NOTIFICATIONINST_GETJSON': MethodDec('X.1nR', 'A01'),
    'NOTIFICATION_SHOW': MethodDec('X.2dM', 'A01', 'android.content.Context', 'java.lang.CharSequence', 'int'),
    'PHOTOTIMERCONTROLLER_INITPICTURECOUNTDOWN': MethodDec('X.2yZ', 'A01'),
    'POSTDETAIL_SETDATA': MethodDec('X.1ed', 'A05', skip_params=True),
    'POSTGESTURE_SIMPLEPOST_ONDOUBLETAP': MethodDec('X.239', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTGESTURE_CAROUSELIMAGE_ONDOUBLETAP': MethodDec('X.2CQ', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTGESTURE_CAROUSELVIDEO_ONDOUBLETAP': MethodDec('X.2WQ', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTVIEW_ADDMEDIA': MethodDec('X.1ei', 'A01', skip_params=True),
    'PROFILEDETAIL_SETDATA': MethodDec('X.3B2', 'A02', skip_params=True),
    'PROFILE_ISSELF': MethodDec('X.0lY', 'A04', 'X.0Ne', 'X.0k0'),
    'PROFILE_ONLONGCLICK': MethodDec('X.3B9', 'A00', skip_params=True),
    'PROXYGEN_SENDREQUESTWITHBODY': MethodDec('com.facebook.proxygen.JniHandler', 'sendRequestWithBodyAndEom', 'org.apache.http.client.methods.HttpUriRequest', 'byte[]', 'int', 'int'),
    'REELVIDEOPLAYER_SEEKTO': MethodDec('X.2MU', 'A0U', 'int'),
    'REELVIEWERFRAGMENT_ADVANCETONEXTREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'BQN', 'java.lang.Object'),
    'REELVIEWERFRAGMENT_PAUSEREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0f', 'com.instagram.reels.fragment.ReelViewerFragment', 'java.lang.String'),
    'REELVIEWERFRAGMENT_RESUMEREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0M', 'com.instagram.reels.fragment.ReelViewerFragment'),
    'REELVIEWERFRAGMENT_UPDATEMEDIA': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0p', 'X.1pp', 'X.1vm'),
    'RESPONSE_HANDLER_ONRESPONSE': MethodDec('X.0ns', 'A00', 'X.0nr', 'X.1Ja'),
    'REGULARNOTIFICATION_RUN': MethodDec('X.BiQ', 'run'),
    'SERIALSCHEDULER_EXECUTE': MethodDec('X.0iw', 'A02', 'X.0pz'),
    'SESSION_GETTOKEN': MethodDec('X.0Ne', 'getToken'),
    'SIMPLEEXECUTOR_ONFAIL': MethodDec('X.4QM', 'onFail', 'X.1vI'),
    'SIMPLEEXECUTOR_ONSUCCESS': MethodDec('X.4QM', 'onSuccess', 'java.lang.Object'),
    'SIMPLEXECUTOR_CONSTRUCTOR': MethodDec('X.4QM', '<init>', 'X.0Ne'),
    'SWIPENAVIGATIONCONTAINER_ONTOUCHEVENT': MethodDec('com.instagram.ui.swipenavigation.SwipeNavigationContainer', 'onTouchEvent', 'android.view.MotionEvent'),
    'USER_JSON_GENERATE': MethodDec('X.1os', 'A03', 'X.0kE', 'X.0jz'),
    'VIDEOSETTINGSMANAGER_ISAUTOPLAYDISABLED': MethodDec('X.1hW', 'A00', 'X.0Ne'),
    'VOLUMEINDICATOR_SETPROGRESS': MethodDec('com.instagram.ui.widget.volume.VolumeIndicator', 'A00', 'int', 'int')
}

In [7]:
named_f_decs = {
    'ACTIONBAR_BASEVIEW': FieldDec('X.1LR', 'A09'),
    'CLIPSHOLDER_MEDIA': FieldDec('X.2BL', 'A00'),
    'COMMENTTOUCHHANDLER_COMMENT': FieldDec('X.3Sh', 'A04'),
    'COMMENT_TEXT': FieldDec('X.1pS', 'A0a'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_DMHOLDER': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_ROOTVIEW': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'mRootView'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_VIDEOPLAYER': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'mVideoPlayer'),
    'DMGESTUREDETECTOR_MESSAGEOBJECT': FieldDec('X.3KP', 'A01'),
    'DMHOLDER_MEDIA': FieldDec('X.5GX', 'A07'),
    'DMVIDEOPLAYER_CONTROLLER': FieldDec('X.5Ga', 'A04'),
    'DMVOICEMESSAGE_RAWMEDIA': FieldDec('X.53g', 'A04'),
    'FEEDITEM_TYPE': FieldDec('X.1vG', 'A0I'),
    'HTTPREQUESTTASK_EXECUTOR': FieldDec('X.0zQ', 'A00'),
    'IGTVVIEWERFRAGMENT_ROOTVIEW': FieldDec('X.9Uw', 'A0Y'),
    'MAINFEED_SESSION': FieldDec('X.1RC', 'A0Y'),
    'MEDIANOTIFICATION_LIST': FieldDec('X.1vC', 'A06'),
    'MEDIA_WRAPPER_LIVEMEDIAVAR': FieldDec('X.1vm', 'A0A'),
    'MEDIA_WRAPPER_REELMEDIAVAR': FieldDec('X.1vm', 'A09'),
    'NATIVEINPUTSTREAM_READBUFFER': FieldDec('X.1Ks', 'A02'),
    'NATIVEINPUTSTREAMHOLDER_NATIVEINPUTSTREAM': FieldDec('X.1n0', 'A01'),
    'NOTIFICATION_INSTANCE': (FieldDec('X.BiP', 'A00'), FieldDec('X.BiQ', 'A00')),
    'NOTIFICATION_SESSION': (FieldDec('X.BiP', 'A01'), FieldDec('X.BiQ', 'A01')),
    'POSTDETAILUI_PROFILENAME': FieldDec('X.1tV', 'A06'),
    'POST_VIEW_COMMENTBUTTON': FieldDec('X.1tY', 'A02'),
    'PROFILEDETAILUI_BIOTEXT': FieldDec('X.3B5', 'A0N'),
    'PROFILEDETAILUI_FOLLOWER': FieldDec('X.3B5', 'A05'),
    'PROFILEDETAILUI_FOLLOWING': FieldDec('X.3B5', 'A06'),
    'PROFILEDETAILUI_FULLNAME': FieldDec('X.3B5', 'A0A'),
    'PROFILE_PROFILEPIC': FieldDec('X.3B7', 'A09'),
    'PROXYGEN_PROXYGENRESPONSEHANDLER': FieldDec('com.facebook.proxygen.JniHandler', 'mResponseHandler'),
    'PROXYGENRESPONSEHANDLER_READBUFFER': FieldDec('X.1Kq', 'A09'),
    'REELVIDEOPLAYER_CONTROLLER': FieldDec('X.2yh', 'A0D'),
    'REEL_VIEWER_FRAGMENT_SESSION': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'A1G'),
    'REEL_VIEWER_FRAGMENT_VIDEOPLAYER': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'mVideoPlayer'),
    'REEL_VIEWER_FRAGMENT_VIEWROOT': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'mViewRoot'),
    'RESPONSEHANDLER_NATIVEINPUTSTREAMHOLDER': FieldDec('X.1mz', 'A00'),
    'STORYUSER_SESSION': FieldDec('X.1vm', 'A0E')
}

In [8]:
m_decs = tuple(named_m_decs.values())
f_decs = []
for f_dec in named_f_decs.values():
    if isinstance(f_dec, Iterable):
        f_decs.extend(f_dec)
    else:
        f_decs.append(f_dec)
f_decs = tuple(f_decs)

# Add Classes of Methods and Fields to c_decs
c_decs = tuple(map(lambda m: m.class_name, m_decs)) \
        + tuple(map(lambda v: v.class_name, f_decs)) + tuple(named_c_decs.values())

In [9]:
# Without using Akrolyb's Auto-Replacing Values:

# c_decs = tuple()
# m_decs = tuple()
# f_decs = tuple()

# Processing and Matching

Loading the accumulator, an object that manages all the possible candidates that are matched by the different Matchers, and extracts the matching candidates. It also performs Inner joins on previous candidates to find the exact (or optimal) match.

In [10]:
accumulator = Accumulator()

Resolving the previously defined MethodDecs. If this fails, the MethodDecs are wrong and contain an error. Make sure the method specified with the MethodDec exists.

In [11]:
dec_cas = resolve_classes(dx, c_decs)

r_cas = tuple(dec_cas.values())
r_mas = resolve_methods(m_decs, dec_cas)
r_fas = resolve_fields(f_decs, dec_cas)

print("Resolved all Classes, Methods and Fields")
if False:
    print("", *map(pretty_format_ma, r_mas), sep="\n* ")

# Arguments to provide to Strategies
s_args = (dx, dx2, r_cas, r_mas, r_fas, accumulator)

Resolved all Classes, Methods and Fields


## Strategies & StrategyHandler

Strategies are essentially functions returning candidates for Classes, Methods or Fields. A `strategy` gets registered in the `StrategyHandler` with Flags indicating whether it tries to match Classes, Methods or Fields. If a search is invoked for a matching flag, the strategy gets invoked on the given arguments.

Global Parameters and Rules of the Strategies allow for easily changing criteria, e. g. use a more lenient search and not require strict matches.

In [12]:
strategies = StrategyHandler()


strings_strategy.MAX_USAGE_COUNT_STR = 20
strings_strategy.UNIQUE_STRINGS_MAJORITY = 2 / 3

methods_strategy.MIN_MATCH_POINTS = 2

## String Strategy

### Exact Counter Match

Extracts Strings used either in the given methods directly or in the classes the methods define for both, the old version and the new version. It then compares the Counters for classes and methods and tries to find exact matches between the Counter Objects.

In [13]:
def string_counter_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs, candidates_ms = string_s.compare_counters()
    
    accumulator.add_candidates(candidates_cs, candidates_ms)

strategies.add_strategy(string_counter_strategy, FLAG_CLASS | FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

* Found multiple candidates for Method X.9lT#<init>(X.2BL, X.9md, X.0Ne, com.instagram.clips.intf.ClipsViewerConfig, android.view.View, X.9PD, X.6Zo)
+ Found single candidate for Method. Considering it a match 
	com.instagram.direct.visual.DirectVisualMessageViewerController#A04(com.instagram.direct.visual.DirectVisualMessageViewerController) -> com.instagram.direct.visual.DirectVisualMessageViewerController#A04(com.instagram.direct.visual.DirectVisualMessageViewerController)
+ Found single candidate for Method. Considering it a match 
	X.1nR#A01() -> X.1nh#A01()
+ Found single candidate for Method. Considering it a match 
	X.0SW#A00(android.content.Context) -> X.0Se#A00(android.content.Context)
+ Found single candidate for Method. Considering it a match 
	X.1RC#configureActionBar(X.1LS) -> X.1RS#configureActionBar(X.1Li)
+ Found single candidate for Method. Considering it a match 
	com.instagram.reels.fragment.ReelViewerFragment#A0p(X.1pp, X.1vm) -> com.instagram.reels.fragment.ReelVi

### Unique Strings

Gather all Strings that are used only in a single class ("Unique Strings") that we still need to match. Then try to find the matching class by only searching for the Unique Strings.

In [14]:
def unique_strings_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = string_s.compare_unique_strings()
    
    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(unique_strings_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

.. Considering ambiguous match by UniqueString as match with a majority of 0.97 (LX/9Uw; -> LX/9Ym;)
+ Found single candidate for Class. Considering it a match! 
	LX/0Ne; -> LX/0Ne;
+ Found single candidate for Class. Considering it a match! 
	LX/0iw; -> LX/0j8;
+ Found single candidate for Class. Considering it a match! 
	LX/0ru; -> LX/0s6;
+ Found single candidate for Class. Considering it a match! 
	LX/0zQ; -> LX/0zc;
+ Found single candidate for Class. Considering it a match! 
	LX/129; -> LX/12L;
+ Found single candidate for Class. Considering it a match! 
	LX/1Kq; -> LX/1L5;
+ Found single candidate for Class. Considering it a match! 
	LX/1Ks; -> LX/1L7;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/ui/swipenavigation/SwipeNavigationContainer; -> Lcom/instagram/ui/swipenavigation/SwipeNavigationContainer;
+ Found single candidate for Class. Considering it a match! 
	LX/1LR; -> LX/1Lh;
+ Found single candidate for Class. Considering it a match! 
	Land

## Structure Strategy

Iterating through every single class and checks for each unmatched class if both have a similar "Profile":
* Number of Methods and Fields
* Types of Fields and Descriptors of Methods

In [15]:
def structure_strategy(r_cas, r_mas, r_fas):
    structure_s = structures_strategy.StructureStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = structure_s.get_exact_structure_matches()

    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(structure_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Class. Considering it a match! 
	LX/9lT; -> LX/9Sz;
+ Found single candidate for Class. Considering it a match! 
	LX/3Sh; -> LX/3T7;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher; -> Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher;
+ Found single candidate for Class. Considering it a match! 
	LX/3KP; -> LX/3Kp;
+ Found single candidate for Class. Considering it a match! 
	LX/5Bk; -> LX/53B;
+ Found single candidate for Class. Considering it a match! 
	LX/5Uy; -> LX/4RQ;
+ Found single candidate for Class. Considering it a match! 
	LX/0j5; -> LX/0jH;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/feed/media/Media__JsonHelper; -> Lcom/instagram/feed/media/Media__JsonHelper;
+ Found single candidate for Class. Considering it a match! 
	LX/2yZ; -> LX/2yv;
+ Found single candidate for Class. Considering it a match! 
	LX/239; -> LX/23S;
* Found 

## Method Strategy

Uses different weighted criteria to get exact or optimal matches. The criteria are:
* Modifiers
* Return Type and Parameter Types
* Length of Bytecode
* References to and from

In [16]:
def method_strategy(r_cas, r_mas, r_fas):
    method_s = methods_strategy.MethodStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)

    print("Exact Matching")
    print()

    # Exact Matches
    candidates_ms = method_s.try_resolve_ms(True)
    accumulator.add_candidates(candidates_ms=candidates_ms)

    print()
    print("Non-Exact Matching")
    print()

    method_s.update_matched()
    # Non-Exact Matches by using weights on the criteria
    candidates_ms = method_s.try_resolve_ms(False)
    accumulator.add_candidates(candidates_ms=candidates_ms)
    
strategies.add_strategy(method_strategy, FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Exact Matching

Class not resolved for Method X.2CQ#onDoubleTap(android.view.MotionEvent)
Class not resolved for Method X.2WQ#onDoubleTap(android.view.MotionEvent)
+ Found single candidate for Method. Considering it a match 
	X.9lT#<init>(X.2BL, X.9md, X.0Ne, com.instagram.clips.intf.ClipsViewerConfig, android.view.View, X.9PD, X.6Zo) -> X.9Sz#<init>(X.2Be, X.9TZ, X.0Ne, com.instagram.clips.intf.ClipsViewerConfig, android.view.View, X.9N1, X.5nO)
+ Found single candidate for Method. Considering it a match 
	X.3Sh#onLongPress(android.view.MotionEvent) -> X.3T7#onLongPress(android.view.MotionEvent)
+ Found single candidate for Method. Considering it a match 
	com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#loadAndLaunchDeveloperOptions(android.content.Context, X.1Gp, androidx.fragment.app.FragmentActivity, X.0Ne, android.os.Bundle) -> com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#loadAndLaunchDeveloperOptions(android.content.Context, X.1H2, androidx.fragment.app

+ Found single candidate for Method. Considering it a match 
	X.3B2#A02(X.3B5, android.content.Context, X.0Ne, X.0jz, X.0TU, com.instagram.profile.fragment.UserDetailDelegate, boolean, X.1f4, java.lang.Integer, boolean, boolean, X.1i2) -> X.3BS#A02(X.3BV, android.content.Context, X.0Ne, X.0kB, X.0Tg, com.instagram.profile.fragment.UserDetailDelegate, boolean, X.1fK, java.lang.Integer, boolean, boolean, X.1iI)
+ Found single candidate for Method. Considering it a match 
	X.0Ne#getToken() -> X.0Ne#getToken()
+ Found single candidate for Method. Considering it a match 
	X.1os#A03(X.0kE, X.0jz) -> X.1p8#A03(X.0kQ, X.0kB)
Could resolve 0 new Classes, 6 new Methods, 0 new Fields.


## Field Strategy

Gathers all fields of matching Class and filters using the following criteria:
* By Type if a matching Type has been found
* By weighted sub-criteria such as:
  * Access Flags
  * Size
  * Number of reads & Number of writes
* By Index if the list of Fields has not changed dramatically

In [17]:
def field_strategy(r_cas, r_mas, r_fas):
    field_s = fields_strategy.FieldStrategy(*s_args)
    
    print("Resolving Types of Fields")
    strategies.invoke_strategies(tuple(field_s.get_types_to_match()))
    print()
    
    candidates_fs = field_s.try_resolve_fs()
    
    accumulator.add_candidates(candidates_fs=candidates_fs)
    
strategies.add_strategy(field_strategy, FLAG_FIELD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Resolving Types of Fields
Could resolve 0 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/0jz; -> LX/0kB;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/common/ui/widget/imageview/IgImageView; -> Lcom/instagram/common/ui/widget/imageview/IgImageView;
+ Found single candidate for Class. Considering it a match! 
	LX/1vM; -> LX/1vf;
+ Found single candidate for Class. Considering it a match! 
	LX/2DQ; -> LX/2Dj;
Could resolve 4 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/14R; -> LX/14d;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/ui/gesture/GestureManagerFrameLayout; -> Lcom/instagram/ui/gesture/GestureManagerFrameLayout;
+ Found single candidate for Class. Considering it a match! 
	Lcom/facebook/proxygen/ReadBuffer; -> Lcom/facebook/proxygen/ReadBuffer;
+ Found single candidate for Class. Considering it a 

# Results
Display summary and matching pairs.

Output to compatible MethodDec input to update from auto-updated values.

In [18]:
print(len(accumulator.matching_cs), "/", len(c_decs), "Classes were matched")
print(len(accumulator.matching_ms), "/", len(m_decs), "Methods were matched")
print(len(accumulator.matching_fs), "/", len(f_decs), "Fields were matched")

print()
print("Matching Classes:")
for c1, c2 in accumulator.matching_cs.items():
    print("•", pretty_format_class(c1), "->", pretty_format_class(c2))

print()
print("Matching Methods:")
for m, ma in accumulator.matching_ms.items():
    print("•", pretty_format_ma(m), "->", pretty_format_ma(ma))

print()
print("Matching Fields:")
for fa, fa2 in accumulator.matching_fs.items():
    print("•", pretty_format_fa(fa), "->", pretty_format_fa(fa2))

74 / 145 Classes were matched
56 / 60 Methods were matched
37 / 39 Fields were matched

Matching Classes:
• com.instagram.direct.visual.DirectVisualMessageViewerController -> com.instagram.direct.visual.DirectVisualMessageViewerController
• X.1nR -> X.1nh
• X.0SW -> X.0Se
• X.1RC -> X.1RS
• com.instagram.reels.fragment.ReelViewerFragment -> com.instagram.reels.fragment.ReelViewerFragment
• com.instagram.mainactivity.MainActivity -> com.instagram.mainactivity.MainActivity
• com.instagram.realtimeclient.RealtimeClientManager -> com.instagram.realtimeclient.RealtimeClientManager
• X.4Ij -> X.4JA
• X.0ns -> X.0o4
• X.1hW -> X.1hm
• X.2MU -> X.2Mn
• X.4QM -> X.4Qx
• X.BiP -> X.Afi
• X.BiQ -> X.Afj
• X.0Ne -> X.0Ne
• X.0iw -> X.0j8
• X.0ru -> X.0s6
• X.0zQ -> X.0zc
• X.129 -> X.12L
• X.1Kq -> X.1L5
• X.1Ks -> X.1L7
• com.instagram.ui.swipenavigation.SwipeNavigationContainer -> com.instagram.ui.swipenavigation.SwipeNavigationContainer
• X.1LR -> X.1Lh
• androidx.fragment.app.Fragment -> andro

• com.instagram.reels.fragment.ReelViewerFragment#A0f(com.instagram.reels.fragment.ReelViewerFragment, java.lang.String) -> com.instagram.reels.fragment.ReelViewerFragment#A0f(com.instagram.reels.fragment.ReelViewerFragment, java.lang.String)
• com.instagram.reels.fragment.ReelViewerFragment#A0M(com.instagram.reels.fragment.ReelViewerFragment) -> com.instagram.reels.fragment.ReelViewerFragment#A0M(com.instagram.reels.fragment.ReelViewerFragment)
• X.0iw#A02(X.0pz) -> X.0j8#A02(X.0qB)
• X.4QM#<init>(X.0Ne) -> X.4Qx#<init>(X.0Ne)
• com.instagram.ui.swipenavigation.SwipeNavigationContainer#onTouchEvent(android.view.MotionEvent) -> com.instagram.ui.swipenavigation.SwipeNavigationContainer#onTouchEvent(android.view.MotionEvent)
• com.instagram.ui.widget.volume.VolumeIndicator#A00(int, int) -> com.instagram.ui.widget.volume.VolumeIndicator#A00(int, int)
• androidx.fragment.app.Fragment#getActivity() -> androidx.fragment.app.Fragment#getActivity()
• X.2DP#A00(X.0kE, X.2DQ) -> X.2Di#A00(X.0kQ,

In [19]:
generate.generate_m_decs(m_decs, r_mas, accumulator.matching_ms)
generate.generate_c_decs(c_decs, r_cas, accumulator.matching_cs)
generate.generate_f_decs(f_decs, r_fas, accumulator.matching_fs)

m_decs = (
    MethodDec('com.instagram.mainactivity.MainActivity', 'onDestroy'),
    MethodDec('X.9Sz', '<init>', 'X.2Be', 'X.9TZ', 'X.0Ne', 'com.instagram.clips.intf.ClipsViewerConfig', 'android.view.View', 'X.9N1', 'X.5nO'),
    MethodDec('X.3T7', 'onLongPress', 'android.view.MotionEvent'),
    MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.1H2', 'androidx.fragment.app.FragmentActivity', 'X.0Ne', 'android.os.Bundle'),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchProjectEncoreSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Ne),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Ne),
    MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0C', 'com.instagr

    'com.instagram.realtimeclient.RealtimeClientManager'
    'X.12L'
    'X.0jH'
    'X.2Di'
    'X.1RS'
    'X.1fK'
    'com.instagram.feed.media.Media__JsonHelper'
    'X.1nh'
    'X.2di'
    'X.2yv'
    'X.1et'
    'X.23S'
    # No Match Found for Class 'X.2CQ'
    # No Match Found for Class 'X.2WQ'
    'X.1ey'
    'X.0lk'
    'X.3BS'
    'X.3BZ'
    'com.facebook.proxygen.JniHandler'
    'X.2Mn'
    'com.instagram.reels.fragment.ReelViewerFragment'
    'X.0o4'
    'X.Afj'
    'X.0j8'
    'X.0Ne'
    'X.4Qx'
    'com.instagram.ui.swipenavigation.SwipeNavigationContainer'
    'X.1p8'
    'X.1hm'
    'com.instagram.ui.widget.volume.VolumeIndicator',
)
f_decs = (
    FieldDec('X.1Lh', 'A09'),
    FieldDec('X.2Be', 'A00'),
    FieldDec('X.3T7', 'A04'),
    FieldDec('X.1pi', 'A0a'),
    FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D'),
    FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'mRootView'),
    FieldDec('com.instagra

# Warnings

Shows some warnings that result in common bugs, so that the Hooks (or other kind of modifications applied) must be updated logic-wise. 

Currently supports: 
* Change in number of parameters 

In [20]:
for m_old, m_new in accumulator.matching_ms.items():
    def get_number_of_params(m):
        return len(tuple(get_pretty_params(str(m.get_descriptor()))))
    
    if get_number_of_params(m_old) != get_number_of_params(m_new):
        print("Warning: Number of parameters changed for", pretty_format_ma(m_old))

# Interoptability with Akrolyb

GraphGuard was build with Akrolyb in mind and has full Akrolyb support, meaning that it is not only capable of automatically matching the given declarations, but also automatically editing the "declaration file" to update matched declarations and mark unmatched items with a `/* TODO */` comment.

The following Code snippet allows to "extract" MethodDecs from MemberDeclarations in Akrolyb. This automates the process of providing GraphGuard the hooks it should find. See [this](#List-of-Classes,-Methods-and-Fields) to learn how to use GraphGuard's Akrolyb Support.

## Extract

It is not in Python, since it would require a Kotlin Parser and evaluating imports, variables etc. Just executing the Constructors in Kotlin, then getting the values is much easier than static analysis. 

The `main` function can (and should) be run statically (locally on the machine and not on your Android) to build the list of `MethodDec`s that GraphGuard is supposed to match in an updated build of the target application. Adjust the MemberDeclarations Class to the Class where you declared the `MemberDeclaration`s.


```kotlin
fun main() {
    GraphGuard.printGeneratedDecs(ClassDecs, MemberDecs, VariableDecs)
}
```

In [21]:
file_dir = "/home/jaqxues/CodeProjects/IdeaProjects/SnipTools/"
c_file = file_dir + "packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/ClassDeclarations.kt"
m_file = file_dir + "packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/MemberDeclarations.kt"
f_file = "/dev/null"


file_dir = "/home/jaqxues/CodeProjects/IdeaProjects/Instaprefs/"
c_file = file_dir + "packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/ClassDecsV8.kt"
m_file = file_dir + "packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/MemberDecsV8.kt"
f_file = file_dir + "packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/VariableDecsV8.kt"

In [22]:
%autoreload
c_txt = io_akrolyb.replace_cs(c_file, accumulator)
m_txt = io_akrolyb.replace_ms(m_file, accumulator, named_m_decs)
f_txt = io_akrolyb.replace_fs(f_file, accumulator, named_f_decs)

No matching Method found for com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchProjectEncoreSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Ne)
No matching Method found for com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Ne)
No matching Method found for X.2CQ#onDoubleTap(android.view.MotionEvent)
No matching Method found for X.2WQ#onDoubleTap(android.view.MotionEvent)
No matching Field found for X.1tV#A06
No matching Field found for X.1tY#A02


In [23]:
print(c_txt)
print(m_txt)
print(f_txt)

package com.marz.instaprefs.packimpl.decs.armv8

import com.jaqxues.akrolyb.genhook.decs.ClassDec
import com.marz.instaprefs.packimpl.decs.AClassDecs

object ClassDecsV8: AClassDecs() {
    override val MAIN_ACTIVITY = ClassDec("com.instagram.mainactivity.MainActivity")

    override val CLIPITEM = ClassDec("X.9Sz")

    override val COMMENT_LONGCLICK_HANDLER = ClassDec("X.3T7")

    override val DEVELOPEROPTIONSLAUNCHER = ClassDec("com.instagram.debug.devoptions.api.DeveloperOptionsLauncher")

    override val DIRECT_VISUAL_MESSAGE_VIEW_CONTROLLER = ClassDec("com.instagram.direct.visual.DirectVisualMessageViewerController")

    override val DM_GESTUREDETECTOR = ClassDec("X.3Kp")

    override val DM_HOLDER = ClassDec("X.53B")

    override val DM_REPLY = ClassDec("X.4JA")

    override val DM_VOICEMESSAGE = ClassDec("X.56X")

    override val DMVIDEOPLAYER = ClassDec("X.4RQ")

    override val FEEDITEM = ClassDec("X.1vZ")

    override val FRAGMENT = ClassDec("androidx.fragment.app.F

}
package com.marz.instaprefs.packimpl.decs.armv8

import android.view.ViewGroup
import android.widget.FrameLayout
import android.widget.ImageView
import android.widget.LinearLayout
import android.widget.TextView
import com.jaqxues.akrolyb.genhook.decs.FieldClass
import com.jaqxues.akrolyb.genhook.decs.VariableDec
import com.marz.instaprefs.packimpl.decs.AVariableDecs
import com.marz.instaprefs.packimpl.decs.armv8.ClassDecsV8.DM_GESTUREDETECTOR
import java.io.InputStream

object VariableDecsV8: AVariableDecs() {
    @FieldClass(["X.1Lh"])
    override val ACTIONBAR_BASEVIEW = VariableDec<ViewGroup>("A09")

    @FieldClass(["X.2Be"])
    override val CLIPSHOLDER_MEDIA = VariableDec<Any>("A00")

    @FieldClass(["X.3T7"])
    override val COMMENTTOUCHHANDLER_COMMENT = VariableDec<Any?>("A04")

    @FieldClass(["X.1pi"])
    override val COMMENT_TEXT = VariableDec<String>("A0a")

    @FieldClass(["com.instagram.direct.visual.DirectVisualMessageViewerController"])
    override val DIRECTVI